In [1]:
import random
import pandas as pd
import numpy as np
import Covid19_model as CM
import matplotlib.pyplot as plt
import numpy as np
import shutil
import sys
import os.path
import math
import csv
from pyomo.environ import *
import lhsmdu
import time

# Set parameter

In [2]:
D = 1e2
n_site = 8
pop = np.array(pd.read_csv("population_"+str(n_site)+".csv",header=None))
pop = np.reshape(np.ones((6, 1)) * pop, (6, n_site))
init_state = np.array(pd.read_csv("state_"+str(n_site)+".csv",header=None))
density = np.array(pd.read_csv("density_"+str(n_site)+".csv",header=None))
dij = np.array(pd.read_csv("distance_"+str(n_site)+".csv",header=None)) / D
N = np.sum(init_state)
n_state = 7 * n_site + 1
n_action = 5 * n_site
density = np.reshape(density, (1, n_site))
T = 90
budget = 1.8e6

In [3]:
deter_para = dict(N=N, P=pop, B=budget, T=T, D=dij, site=n_site, density=density,
                  gamma=0.69, alpha=0.6, v_max = 0.2/14,
                  cost_ti=0.0977, cost_ta=0.02, cost_v=0.07, 
                  cost_poc_0=0.000369, cost_poc_1=0.001057, 
                  pid=1.10/1000, psr=0.7/3, pid_plus=0.1221/1000, pir=1/8, prs=1/180)

in_deter_para = dict(beta2=[0.15204, 0.16287], beta1=[0.7602, 0.81435],
                     pei=[0.07143, 0.14286], per=[0.04000,0.05556])

in_deter_truth = dict(beta2=0.15747, beta1=0.78735, pei=0.10714, per=0.04545)

para_truth = {**deter_para, **in_deter_truth}

In [4]:
seed = 0
env = CM.Env_model(init_state, deter_para, in_deter_para, seed)
env.set_para_truth(para_truth)

In [5]:
tic = time.time()
random.seed(1)
observation, state = init_state.copy(), init_state.copy()
state_buffer, new_buffer  = np.zeros([T, 6]), np.zeros([T, 4])
score, ni, ne, num_sample = 0, 0, 0, 20000
start_time = 6
done = False
index = [0]

for t in range(T):
    pset = env.para_sampling(100)
    para = pset[index[0]]
    action_buffer = [] 
    reward_buffer = []
    
    if budget > 0 and t > start_time:     
        for j in range(num_sample):  
            b0 = budget
            s0 = score
            ac = [random.random(),random.random(),random.random(),random.random(),random.random()]
            ac = [ac * n_site]
            ac = np.reshape(np.array(ac), (n_site, 5)).T
            
            action_buffer.append(ac) 
            state_ex, reward, _, _, c = env.transition(state, ac, para)  
            s0 += reward
            b0 -= c
            for i in range(t, T):
                if b0 > 0:
                    act = np.ones((5, n_site)) 

                else:
                    act = np.zeros((5, n_site))
                next_ex, reward_ex, _, _, c = env.transition(state_ex, act, para)  
                s0 += reward_ex
                b0 -= c
                state_ex = next_ex.copy()          
            reward_buffer.append(s0)
               
        max_reward = np.argmax(np.array(reward_buffer))
        action = action_buffer[max_reward]
    else:
        action = np.zeros((5, n_site))
    next_state, obs, reward, cost, new_i, new_e, ck = env.online_state_estimate(state, observation, 
                                                                                action, pset, 100, 1)

    index = ck.argsort()[0][:1]
    score += reward
    budget -= cost
    ni += new_i
    ne += new_e
    new_buffer[t, 0], new_buffer[t, 1], new_buffer[t, 2], new_buffer[t, 3] = new_i, ni, new_e, ne
    print('\r', 'Time: {} | S: {:.1f}, E: {:.1f}, A: {:.1f}, I: {:.1f}, D: {:.1f}, R: {:.1f}, Budget: {:.2f}, reward:{:.2f}'
          .format(t, np.sum(observation[0]), np.sum(observation[1]), np.sum(observation[2]), 
                  np.sum(observation[3]), np.sum(observation[4]), np.sum(observation[5]), 
                  budget, score))
    print('\r', 'Time: {} | S: {:.1f}, E: {:.1f}, A: {:.1f}, I: {:.1f}, D: {:.1f}, R: {:.1f}, Budget: {:.2f}, reward:{:.2f}'
          .format(t, np.sum(state[0]), np.sum(state[1]), np.sum(state[2]), np.sum(state[3]), 
                  np.sum(state[4]), np.sum(state[5]), budget, score))
    print('\r', 'Time: {} | new_e:{:.1f}, cum_e:{:.1f}, new_i:{:.1f}, cum_i:{:.1f}'.format(t, new_e, ne, new_i, ni))
    print(action)
    state_buffer[t, :] = sum(observation.T)
    state = next_state.copy()
    observation = obs.copy()
print(time.time() - tic)

 Time: 0 | S: 18118944.0, E: 56.0, A: 0.0, I: 0.0, D: 0.0, R: 0.0, Budget: 1800000.00, reward:-114.64
 Time: 0 | S: 18118944.0, E: 56.0, A: 0.0, I: 0.0, D: 0.0, R: 0.0, Budget: 1800000.00, reward:-114.64
 Time: 0 | new_e:26.5, cum_e:26.5, new_i:23.6, cum_i:23.6
[[0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]]
 Time: 1 | S: 18118899.9, E: 73.9, A: 0.0, I: 23.6, D: 0.0, R: 2.5, Budget: 1800000.00, reward:-277.18
 Time: 1 | S: 18118901.0, E: 72.0, A: 0.0, I: 23.6, D: 0.0, R: 0.0, Budget: 1800000.00, reward:-277.18
 Time: 1 | new_e:37.1, cum_e:63.6, new_i:32.7, cum_i:56.3
[[0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]]
 Time: 2 | S: 18118838.0, E: 99.8, A: 0.0, I: 56.3, D: 0.0, R: 5.9, Budget: 1800000.00, reward:-508.20
 Time: 2 | S: 18118841.0, E: 100.0, A: 0.0, I: 56.3, D: 0.0, R: 0.0, Budget: 1800000.00, reward:-50

 Time: 14 | S: 17996817.5, E: 201.1, A: 449.8, I: 1110.8, D: 3.7, R: 120417.2, Budget: 1633526.18, reward:-7145.89
 Time: 14 | S: 17996875.0, E: 198.0, A: 449.8, I: 1110.8, D: 3.7, R: 120415.0, Budget: 1633526.18, reward:-7145.89
 Time: 14 | new_e:98.3, cum_e:1591.1, new_i:135.3, cum_i:1760.9
[[0.80736012 0.80736012 0.80736012 0.80736012 0.80736012 0.80736012
  0.80736012 0.80736012]
 [0.63503064 0.63503064 0.63503064 0.63503064 0.63503064 0.63503064
  0.63503064 0.63503064]
 [0.99686094 0.99686094 0.99686094 0.99686094 0.99686094 0.99686094
  0.99686094 0.99686094]
 [0.96604569 0.96604569 0.96604569 0.96604569 0.96604569 0.96604569
  0.96604569 0.96604569]
 [0.95162914 0.95162914 0.95162914 0.95162914 0.95162914 0.95162914
  0.95162914 0.95162914]]
 Time: 15 | S: 17948889.6, E: 106.5, A: 543.3, I: 1157.3, D: 4.2, R: 168299.0, Budget: 1628338.93, reward:-7417.46
 Time: 15 | S: 17948954.0, E: 101.0, A: 543.3, I: 1157.3, D: 4.2, R: 168295.0, Budget: 1628338.93, reward:-7417.46
 Time: 15 

 Time: 25 | S: 17747477.1, E: 103.7, A: 425.7, I: 1205.2, D: 9.1, R: 369779.3, Budget: 1454464.51, reward:-10854.51
 Time: 25 | S: 17747575.0, E: 103.0, A: 425.7, I: 1205.2, D: 9.1, R: 369773.0, Budget: 1454464.51, reward:-10854.51
 Time: 25 | new_e:62.4, cum_e:2365.4, new_i:98.3, cum_i:2918.4
[[0.07423359 0.07423359 0.07423359 0.07423359 0.07423359 0.07423359
  0.07423359 0.07423359]
 [0.28677948 0.28677948 0.28677948 0.28677948 0.28677948 0.28677948
  0.28677948 0.28677948]
 [0.97008424 0.97008424 0.97008424 0.97008424 0.97008424 0.97008424
  0.97008424 0.97008424]
 [0.9232973  0.9232973  0.9232973  0.9232973  0.9232973  0.9232973
  0.9232973  0.9232973 ]
 [0.06708831 0.06708831 0.06708831 0.06708831 0.06708831 0.06708831
  0.06708831 0.06708831]]
 Time: 26 | S: 17745035.9, E: 144.4, A: 366.6, I: 1259.3, D: 10.1, R: 372183.8, Budget: 1411175.77, reward:-11194.51
 Time: 26 | S: 17745136.0, E: 144.0, A: 366.6, I: 1259.3, D: 10.1, R: 372176.0, Budget: 1411175.77, reward:-11194.51
 Time:

 Time: 36 | S: 17613512.1, E: 167.5, A: 228.9, I: 1015.7, D: 14.5, R: 504061.3, Budget: 1324893.04, reward:-13894.24
 Time: 36 | S: 17613609.0, E: 168.0, A: 228.9, I: 1015.7, D: 14.5, R: 504064.0, Budget: 1324893.04, reward:-13894.24
 Time: 36 | new_e:113.4, cum_e:3001.6, new_i:118.1, cum_i:3832.5
[[0.00353927 0.00353927 0.00353927 0.00353927 0.00353927 0.00353927
  0.00353927 0.00353927]
 [0.85386994 0.85386994 0.85386994 0.85386994 0.85386994 0.85386994
  0.85386994 0.85386994]
 [0.7913906  0.7913906  0.7913906  0.7913906  0.7913906  0.7913906
  0.7913906  0.7913906 ]
 [0.84390681 0.84390681 0.84390681 0.84390681 0.84390681 0.84390681
  0.84390681 0.84390681]
 [0.03406421 0.03406421 0.03406421 0.03406421 0.03406421 0.03406421
  0.03406421 0.03406421]]
 Time: 37 | S: 17615915.6, E: 250.6, A: 198.8, I: 1025.1, D: 14.8, R: 501595.2, Budget: 1289653.70, reward:-14424.44
 Time: 37 | S: 17616012.0, E: 244.0, A: 198.8, I: 1025.1, D: 14.8, R: 501600.0, Budget: 1289653.70, reward:-14424.44
 T

 Time: 47 | S: 17459513.7, E: 104.5, A: 348.9, I: 1031.1, D: 18.5, R: 657983.3, Budget: 1121237.19, reward:-17692.84
 Time: 47 | S: 17459606.0, E: 109.0, A: 348.9, I: 1031.1, D: 18.5, R: 657996.0, Budget: 1121237.19, reward:-17692.84
 Time: 47 | new_e:57.3, cum_e:3824.8, new_i:86.8, cum_i:4964.8
[[0.40869385 0.40869385 0.40869385 0.40869385 0.40869385 0.40869385
  0.40869385 0.40869385]
 [0.99027119 0.99027119 0.99027119 0.99027119 0.99027119 0.99027119
  0.99027119 0.99027119]
 [0.98771201 0.98771201 0.98771201 0.98771201 0.98771201 0.98771201
  0.98771201 0.98771201]
 [0.91577854 0.91577854 0.91577854 0.91577854 0.91577854 0.91577854
  0.91577854 0.91577854]
 [0.9544388  0.9544388  0.9544388  0.9544388  0.9544388  0.9544388
  0.9544388  0.9544388 ]]
 Time: 48 | S: 17439288.3, E: 61.4, A: 380.1, I: 990.1, D: 18.6, R: 678261.4, Budget: 1110122.81, reward:-17865.84
 Time: 48 | S: 17439379.0, E: 61.0, A: 380.1, I: 990.1, D: 18.6, R: 678272.0, Budget: 1110122.81, reward:-17865.84
 Time: 4

 Time: 58 | S: 17318527.4, E: 58.1, A: 203.8, I: 785.5, D: 22.4, R: 799402.7, Budget: 956387.77, reward:-19458.00
 Time: 58 | S: 17318631.0, E: 57.0, A: 203.8, I: 785.5, D: 22.4, R: 799415.0, Budget: 956387.77, reward:-19458.00
 Time: 58 | new_e:29.7, cum_e:4177.2, new_i:47.9, cum_i:5580.4
[[0.25956468 0.25956468 0.25956468 0.25956468 0.25956468 0.25956468
  0.25956468 0.25956468]
 [0.90599434 0.90599434 0.90599434 0.90599434 0.90599434 0.90599434
  0.90599434 0.90599434]
 [0.97354497 0.97354497 0.97354497 0.97354497 0.97354497 0.97354497
  0.97354497 0.97354497]
 [0.98504431 0.98504431 0.98504431 0.98504431 0.98504431 0.98504431
  0.98504431 0.98504431]
 [0.85146181 0.85146181 0.85146181 0.85146181 0.85146181 0.85146181
  0.85146181 0.85146181]]
 Time: 59 | S: 17307934.7, E: 37.0, A: 214.6, I: 744.3, D: 22.6, R: 810046.7, Budget: 940570.75, reward:-19549.87
 Time: 59 | S: 17308039.0, E: 36.0, A: 214.6, I: 744.3, D: 22.6, R: 810059.0, Budget: 940570.75, reward:-19549.87
 Time: 59 | new

 Time: 69 | S: 17195973.5, E: 10.1, A: 101.3, I: 466.1, D: 25.2, R: 922423.8, Budget: 767796.07, reward:-20249.88
 Time: 69 | S: 17196065.0, E: 14.0, A: 101.3, I: 466.1, D: 25.2, R: 922438.0, Budget: 767796.07, reward:-20249.88
 Time: 69 | new_e:5.0, cum_e:4319.5, new_i:15.3, cum_i:5882.1
[[0.31401444 0.31401444 0.31401444 0.31401444 0.31401444 0.31401444
  0.31401444 0.31401444]
 [0.46726876 0.46726876 0.46726876 0.46726876 0.46726876 0.46726876
  0.46726876 0.46726876]
 [0.99784221 0.99784221 0.99784221 0.99784221 0.99784221 0.99784221
  0.99784221 0.99784221]
 [0.99106556 0.99106556 0.99106556 0.99106556 0.99106556 0.99106556
  0.99106556 0.99106556]
 [0.34273752 0.34273752 0.34273752 0.34273752 0.34273752 0.34273752
  0.34273752 0.34273752]]
 Time: 70 | S: 17183090.5, E: 10.6, A: 88.8, I: 453.8, D: 25.5, R: 935330.8, Budget: 733228.35, reward:-20283.02
 Time: 70 | S: 17183179.0, E: 14.0, A: 88.8, I: 453.8, D: 25.5, R: 935345.0, Budget: 733228.35, reward:-20283.02
 Time: 70 | new_e:

 Time: 80 | S: 16934003.4, E: 1.1, A: 27.4, I: 183.6, D: 26.3, R: 1184758.2, Budget: 480934.13, reward:-20408.36
 Time: 80 | S: 16934066.0, E: 0.0, A: 27.4, I: 183.6, D: 26.3, R: 1184779.0, Budget: 480934.13, reward:-20408.36
 Time: 80 | new_e:0.7, cum_e:4344.6, new_i:3.5, cum_i:5968.8
[[0.15340344 0.15340344 0.15340344 0.15340344 0.15340344 0.15340344
  0.15340344 0.15340344]
 [0.81440321 0.81440321 0.81440321 0.81440321 0.81440321 0.81440321
  0.81440321 0.81440321]
 [0.97950418 0.97950418 0.97950418 0.97950418 0.97950418 0.97950418
  0.97950418 0.97950418]
 [0.99911937 0.99911937 0.99911937 0.99911937 0.99911937 0.99911937
  0.99911937 0.99911937]
 [0.48829598 0.48829598 0.48829598 0.48829598 0.48829598 0.48829598
  0.48829598 0.48829598]]
 Time: 81 | S: 16931925.0, E: 1.2, A: 23.7, I: 168.4, D: 26.4, R: 1186855.4, Budget: 474926.13, reward:-20413.41
 Time: 81 | S: 16931989.0, E: 0.0, A: 23.7, I: 168.4, D: 26.4, R: 1186875.0, Budget: 474926.13, reward:-20413.41
 Time: 81 | new_e:0.6